In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os.path
import requests
import re

In [2]:
def scroll_down(driver):
    SCROLL_PAUSE_TIME = 1
    while True:
        last_height = driver.execute_script("return document.body.scrollHeight")
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # try again (can be removed)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)
            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            # check if the page height has remained the same
            if new_height == last_height:
                # if so, you are done
                break
            # if not, move on to the next loop
            else:
                last_height = new_height
                continue
    return driver

In [3]:
def scroll_down_and_wait(driver):
    actions = ActionChains(driver)
    for _ in range(4):
        actions.send_keys(Keys.SPACE).perform()
        time.sleep(1)
    return driver

def scroll_up_and_wait(driver):
    actions = ActionChains(driver)
    for _ in range(4):
        actions.send_keys(Keys.PAGE_UP).perform()
        time.sleep(1)
    return driver

In [4]:
driver = webdriver.Chrome(executable_path='D:/IT/Python/Crawl_data/chromedriver.exe')
driver.get('https://www.etsy.com/')

In [5]:
search_input = driver.find_element_by_xpath('//input[contains(@class,"wt-input wt-input-btn-group__input global-enhancements-search-input-btn-group__input wt-pr-xs-7")]')
search_input.send_keys('dog tshirt')
find_button = driver.find_element_by_xpath('//button[contains(@class,"wt-input-btn-group__btn")]')
find_button.click()
time.sleep(2)

In [6]:
driver = scroll_down(driver)
driver = scroll_up_and_wait(driver)

In [7]:
urls = []
all_items = driver.find_elements_by_xpath('//div[contains(@class,"v2-listing-card position-relative")]')
for item in all_items:
    for url in item.find_elements_by_xpath('//div[contains(@class,"v2-listing-card position-relative")]/a[@href]'):
        if len(urls) < 10:
            urls.append(url.get_attribute('href'))
        else: break

In [8]:
index = 0
for url in urls:
    driver.get(url)
    best_seller = driver.find_element_by_xpath('//div[contains(@class,"wt-align-items-center")]/span[contains(@class,"wt-display-inline-block")]').text
    if best_seller == 'Bestseller':
        print(url)
        link_imgs = driver.find_elements_by_xpath('//ul[contains(@class,"wt-list-unstyled")]/li[contains(@class,"wt-position-absolute")]/img[@src]')
        for img in link_imgs:
            img_src = img.get_attribute('src')
            filename = re.search(r'/([\w]+[.]+[\w_-]+[.](jpg|gif|png))$', img_src)
            if filename:
                if not os.path.exists(os.path.dirname('hinh_esty/' + str(index) + filename.group(0))):
                        os.makedirs(os.path.dirname('hinh_esty/' + str(index) + filename.group(0)))
                with open('hinh_esty/' + str(index) + filename.group(0),'wb') as f:
                    response = requests.get(img_src)
                    f.write(response.content)
    index += 1
    time.sleep(2)

https://www.etsy.com/listing/769379665/dog-rescue-shirt-animal-rescue-shirt?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=dog+tshirt&ref=sc_gallery-1-1&plkey=2925a9f2f6a0a58fdd61a9ea25637828606b4e05%3A769379665&pro=1&col=1
https://www.etsy.com/listing/587070148/dog-dad-t-shirt-gift-dog-shirt-for-men?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=dog+tshirt&ref=sc_gallery-1-2&plkey=1723d1b2f0626ba31f686dd561cd0a91884bf3ab%3A587070148&bes=1&col=1
https://www.etsy.com/listing/822512888/t-shirt-for-women-love-paw-shirt-womens?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=dog+tshirt&ref=sc_gallery-1-3&plkey=c793937e92e960ebfb5f67266a7527ec30a921e4%3A822512888&bes=1
https://www.etsy.com/listing/587070148/dog-dad-t-shirt-gift-dog-shirt-for-men?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=dog+tshirt&ref=sr_gallery-1-1&organic_search_click=1&bes=1&col=1
https://www